# Before getting started

Make sure pixi is up to date by running the following command in your terminal:

```bash
pixi self-update
```

# Start JupyterLab

To get the Jupyter kernel to work, 

1. Run the following commands in the terminal in the ispy-readii directory:
    ```bash
    pixi shell --manifest-path=./pixi.toml

    pixi install

    pixi run make_kernel
    ```
2. In the `Select Kernel` menu at the top right of the notebook, select `Jupyter Kernel` as the source. 

3. Refresh the options and one called `ispy-readii` should appear. Select this option.

# Imports

In [2]:
import numpy as np
from imgtools.io import read_dicom_series
from imgtools.autopipeline import AutoPipeline
import pandas as pd
import subprocess

from pathlib import Path
from readii.loaders import loadSegmentation
from readii.image_processing import flattenImage
from readii.feature_extraction import singleRadiomicFeatureExtraction

# Set Dataset Name

In [3]:
DATASET_NAME = 'ISPY2'

# ~~Run Med-Imagetools crawl~~

~~This creates a file used by READII to associate the images with the corresponding segmentation files.~~

This doesn't work for ISPY2, will be using code from `workflow/scripts/python/solid-BET-FSL.py` instead.

In [ ]:
# mit_input_dir = f"../../rawdata/{DATASET_NAME}"
# mit_output_dir = f"../../procdata/{DATASET_NAME}/mit_output"

# modalities = "MR,SEG"
# # LEAVE THIS ONE
# dry_run = False

# # Set up and run the pipeline
# pipeline = AutoPipeline(input_directory=mit_input_dir,
#                         output_directory=mit_output_dir,
#                         modalities=modalities,
#                         dry_run=dry_run,
#                         show_progress=False) 

# pipeline.run()

# Convert MRI to NIFTI

Insert code from `solid-BET-FSL.py` lines 36-52 here

# Run Bias Correction

Insert code fom `solid-BET-FSL.py` lines 81-94 here

# File path setup

This is an example with a CT scan and RTSTRUCT. Copy the below cell and replace with MRI data paths. Keep variable names the same.

In [5]:
# Path to the DIRECTORY that contains all the CT dcm files
image_dir_path = "../../rawdata/ISPY2/images/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/51000.000000-ISPY2 VOLSER uni-lateral cropped SER-33995/"

# Path to the SEG FILE
segmentation_file_path = "../../rawdata/ISPY2/images/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/51900.000000-ISPY2 VOLSER uni-lateral cropped Analysis Mask-78091/1-1.dcm"

# Load in image and segmentation

In [12]:
# Load the image (CT or MR)
image = read_dicom_series(image_dir_path)

# Load in the segmentation file (this is set up for RTSTRUCT)
# This will return a dictionary of segmentation images, with the keys being the ROI names
seg_image_dict = loadSegmentation(segmentation_file_path, modality = 'SEG')

# Get the segmentation image for the ROI of interest
seg_image = flattenImage(seg_image_dict["VOLSER Analysis Mask"])

# Confirm that loading working correctly

The dimensions of the image and segmenation should match.

In [ ]:
print("Image dimensions: ", image.GetSize())
print("Segmentation dimensions: ", seg_image.GetSize())

## Extract Radiomic Features

In [ ]:
# Pick which config settings to use for PyRadiomics feature extraction
pyradiomics_param_file_path = "../config/default_pyradiomics.yaml"

# Run PyRadiomics feature extraction with REAII function
radiomic_features_dict = singleRadiomicFeatureExtraction(image, seg_image, pyradiomics_param_file_path, randomSeed=10)

# Convert to dataframe
rad_features_df = pd.DataFrame.from_dict(radiomic_features_dict, orient="index").transpose()


# Save out the results of the feature extraction

In [9]:
save_radiomics_path = Path("results/4D-Lung/readii_radiomic_features.csv")
save_radiomics_path.parent.mkdir(parents=True, exist_ok=True)

rad_features_df.to_csv(save_radiomics_path, index=False)